In [1]:
import torch
import torch.nn as nn
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm
import torchvision
from tokenization import Tokenizer

torch.__version__

'1.8.1'

In [2]:
from functools import partial

data_path = "../mjsynth/90kDICT32px"

train_labels = pd.read_csv("preprocessed/annotation_train.csv")
train_labels['path'] = train_labels['path'].apply(lambda x: f'{data_path}/{x}')
train_labels['tgt'] = train_labels['tgt'].astype('str')

In [3]:
train_labels.head()

,path,tgt
0,../mjsynth/90kDICT32px/2425/1/115_Lube_45484.jpg,Lube
1,../mjsynth/90kDICT32px/2425/1/114_Spencerian_7...,Spencerian
2,../mjsynth/90kDICT32px/2425/1/113_accommodatin...,accommodatingly
3,../mjsynth/90kDICT32px/2425/1/112_CARPENTER_11...,CARPENTER
4,../mjsynth/90kDICT32px/2425/1/111_REGURGITATIN...,REGURGITATING


In [4]:
tokenizer = Tokenizer()
tokenizer.vocab_size

{'<PAD>': 0, '<START>': 1, '<END>': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, ' ': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, 'H': 21, 'I': 22, 'J': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'Q': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'W': 36, 'X': 37, 'Y': 38, 'Z': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65}


66

In [5]:
tqdm.pandas()

train_labels['tgt'] = train_labels['tgt'].progress_apply(lambda x: tokenizer.string_to_indices(x))

100%|██████████| 7224612/7224612 [00:34<00:00, 208946.94it/s]


In [6]:
train_labels['tgt_len'] = train_labels['tgt'].progress_apply(lambda x:len(x))

100%|██████████| 7224612/7224612 [00:06<00:00, 1041590.48it/s]


In [7]:
train_labels.head()

,path,tgt,tgt_len
0,../mjsynth/90kDICT32px/2425/1/115_Lube_45484.jpg,"[1, 25, 60, 41, 44, 2]",6
1,../mjsynth/90kDICT32px/2425/1/114_Spencerian_7...,"[1, 32, 55, 44, 53, 42, 44, 57, 48, 40, 53, 2]",12
2,../mjsynth/90kDICT32px/2425/1/113_accommodatin...,"[1, 40, 42, 42, 54, 52, 52, 54, 43, 40, 59, 48...",17
3,../mjsynth/90kDICT32px/2425/1/112_CARPENTER_11...,"[1, 16, 14, 31, 29, 18, 27, 33, 18, 31, 2]",11
4,../mjsynth/90kDICT32px/2425/1/111_REGURGITATIN...,"[1, 31, 18, 20, 34, 31, 20, 22, 33, 14, 33, 22...",15


In [8]:
train_labels.to_pickle('preprocessed/train_labels.pkl', protocol=4)

In [9]:
train_labels['tgt_len'].max(), train_labels['tgt_len'].mean()

(25, 10.130945301976078)

In [ ]:
import pickle

img_list = []
idx_list = []

path_list = list(train_labels['path'])
idx = list(train_labels.index)
np.random.seed(42)
np.random.shuffle(idx)

del train_labels
gc.collect()

part_id = 0
for i in tqdm(idx):
    img = cv2.imread(path_list[i], cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    else:
        img_list.append(img)
        idx_list.append(i)

        if len(idx_list) == 2000000:
            with open(f'preprocessed/train_part_{part_id}.pkl', 'wb') as f:
                pickle.dump((img_list, idx_list), file=f)

            img_list, idx_list = [], []
            part_id += 1